# 01 — Космология: SDSS DR17 + DESI DR1 → унификация → подвыборки N=10³ и N=10⁴ → kNN‑графы

**Задача ВКР:** построить сопоставимые графовые представления для двух физических доменов (космология и квантовые модели), чтобы сравнивать их **геометрию/спектры/топологию** единым пайплайном.

В этом ноутбуке мы делаем космологическую часть **строго под требования квантовых выборок**:  
- формируем подвыборки **ровно** размеров **N=1 000** и **N=10 000**;
- строим kNN‑графы (узлы — объекты каталога, рёбра — ближайшие соседи в 3D‑координатах);
- сохраняем результат в унифицированном формате `GraphData` (таблица узлов + таблица рёбер + метаданные), чтобы во втором ноутбуке **тот же код** работал и для квантовых семейств.

**Артефакты:**  
- нормализованные данные: `data/processed/cosmology/<preset>/...parquet`  
- графы: `outputs/graphs/cosmology/<preset>/<source>/N_<N>/...`


In [ ]:
# 0) Imports + reproducibility

from __future__ import annotations

from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scientific_api.logging import setup_logging, get_logger
from scientific_api.settings import DEFAULT_SEED, COSMO_SAMPLE_SIZES
from scientific_api.storage.paths import ensure_all_dirs, get_repo_root, get_outputs_dir
from scientific_api.pipeline.cosmology_ingest import run as run_cosmology_ingest
from scientific_api.graphs.knn import GraphData, build_knn_edges, save_graphdata

setup_logging("INFO")
logger = get_logger("nb01")

ensure_all_dirs()
ROOT = get_repo_root()
OUT = get_outputs_dir()

print("Repo root:", ROOT)
print("Outputs:", OUT)
print("Sample sizes:", COSMO_SAMPLE_SIZES)


## 1) Запуск ingestion (SDSS DR17 + DESI DR1) по пресетам

Пайплайн делает:
1. загрузку/выгрузку SDSS (через запросы) в CSV;
2. загрузку DESI DR1 LSS clustering FITS;
3. приведение колонок к единой схеме + фильтры по пресету;
4. сохранение в Parquet;
5. выравнивание размеров выборок (matched downsample), чтобы SDSS и DESI сравнивались честно.

**Важно:** имена столбцов исходных таблиц могут отличаться; маппинг колонок для DESI сохраняется в `outputs/ingestion/<preset>/desi_column_mapping.json`.


In [ ]:
# 1) Ingest both presets used in Chapter 1 logic
# We run both low_z and high_z to obtain more diverse cosmology corpora.

PRESETS = ["low_z", "high_z"]
ingest_meta = {}

for preset in PRESETS:
    logger.info(f"Running ingestion for preset={preset}")
    meta = run_cosmology_ingest(preset)
    ingest_meta[preset] = meta
    print(meta["summary"])


## 2) Загрузка нормализованных matched‑выборок и проверка схемы

Единая схема (см. `scientific_api.data_processing.cosmology.schema`):
`source, sample, obj_id, ra_deg, dec_deg, z, x_mpc, y_mpc, z_mpc, weight`.

Мы **не предполагаем**, что сырой SDSS/DESI имеют эти имена — мы проверяем **после нормализации**.


In [ ]:
from scientific_api.data_processing.cosmology.schema import REQUIRED_COLUMNS

def load_matched_points(preset: str, source: str) -> pd.DataFrame:
    path = ROOT / "data" / "processed" / "cosmology" / preset / f"{source}__matched.parquet"
    if not path.exists():
        raise FileNotFoundError(f"Missing matched parquet: {path}")
    df = pd.read_parquet(path).reset_index(drop=True)
    missing = [c for c in REQUIRED_COLUMNS if c not in df.columns]
    if missing:
        raise ValueError(f"{path} missing required columns: {missing}")
    return df

cosmo = {}
for preset in PRESETS:
    cosmo[(preset, "sdss_dr17")] = load_matched_points(preset, "sdss_dr17")
    cosmo[(preset, "desi_dr1")] = load_matched_points(preset, "desi_dr1")

for (preset, source), df in cosmo.items():
    print(f"{preset:6s} {source:9s}  n={len(df):7d}  z-range=({df['z'].min():.3f},{df['z'].max():.3f})")


## 3) Подвыборки фиксированного размера N=1 000 и N=10 000

Ключевое требование для корректного сравнения: **размер графа** должен быть сопоставим между доменами.  
Поэтому здесь мы строим 4 графа на каждый пресет:
- SDSS: N=1k, N=10k  
- DESI: N=1k, N=10k

Подвыборки делаем **детерминированно**: фиксируем seed, сохраняем индексы в `meta`.


In [ ]:
def fixed_sample(df: pd.DataFrame, n: int, seed: int) -> pd.DataFrame:
    if len(df) < n:
        raise ValueError(f"Requested n={n}, but dataset has only {len(df)} rows.")
    rng = np.random.default_rng(seed)
    idx = rng.choice(len(df), size=n, replace=False)
    sub = df.iloc[idx].copy().reset_index(drop=True)
    sub["node_id"] = np.arange(n, dtype=int)
    sub.attrs["sample_indices"] = idx.tolist()
    return sub

samples = {}  # (preset, source, N) -> df
for preset in PRESETS:
    for source in ["sdss_dr17", "desi_dr1"]:
        df = cosmo[(preset, source)]
        for N in COSMO_SAMPLE_SIZES:
            sub = fixed_sample(df, N, seed=DEFAULT_SEED + hash((preset, source, N)) % 10_000)
            samples[(preset, source, N)] = sub
            print(f"sampled {preset} {source} N={N} -> {len(sub)}")


## 4) Построение kNN‑графов и сохранение в `GraphData`

- Узлы: строки DataFrame (координаты `x_mpc,y_mpc,z_mpc`, физические поля, и т.д.)
- Рёбра: `k` ближайших соседей по евклидовой метрике в 3D
- Вес ребра: расстояние (потом можно нормировать/инвертировать при необходимости)

Сохраняем:
`outputs/graphs/cosmology/<preset>/<source>/N_<N>/{nodes.parquet, edges.parquet, meta.json}`


In [ ]:
K = 12  # fixed for the project (no knobs in the notebook)

graph_index_rows = []

for (preset, source, N), df in samples.items():
    coords = df[["x_mpc", "y_mpc", "z_mpc"]].to_numpy(dtype=float)
    edges = build_knn_edges(coords, k=K, metric="euclidean")

    nodes = df.copy()
    # ensure required node_id
    if "node_id" not in nodes.columns:
        nodes["node_id"] = np.arange(len(nodes), dtype=int)

    meta = {
        "domain": "cosmology",
        "preset": preset,
        "source": source,
        "N": int(N),
        "k": int(K),
        "seed": int(DEFAULT_SEED),
        "coord_cols": ["x_mpc", "y_mpc", "z_mpc"],
        "sample_indices": df.attrs.get("sample_indices", None),
    }

    g = GraphData(nodes=nodes, edges=edges, meta=meta)

    out_dir = OUT / "graphs" / "cosmology" / preset / source / f"N_{N}"
    save_graphdata(g, out_dir)

    graph_index_rows.append({
        "domain": "cosmology",
        "preset": preset,
        "source": source,
        "N": N,
        "k": K,
        "path": str(out_dir),
        "n_edges": int(edges.shape[0]),
    })

graph_index = pd.DataFrame(graph_index_rows).sort_values(["preset","source","N"])
graph_index_path = OUT / "graphs" / "graph_index_cosmology.parquet"
graph_index.to_parquet(graph_index_path, index=False)

graph_index


## 5) Визуальная проверка: 3D‑облако и распределение степеней (N=1000)

Мы делаем быструю sanity‑проверку:
- точки не вырожденные;
- kNN‑граф не пустой;
- степень узлов имеет ожидаемый порядок (в среднем около k).

Показываем по одному примеру SDSS и DESI для каждого пресета.


In [ ]:
from scientific_api.graphs.knn import load_graphdata
import networkx as nx

def edges_to_nx(n: int, edges: pd.DataFrame) -> nx.Graph:
    G = nx.Graph()
    G.add_nodes_from(range(n))
    if len(edges) > 0:
        G.add_weighted_edges_from(edges[["source","target","weight"]].itertuples(index=False, name=None))
    return G

for preset in PRESETS:
    fig = plt.figure(figsize=(12, 5))
    for j, source in enumerate(["sdss_dr17", "desi_dr1"]):
        gdir = OUT / "graphs" / "cosmology" / preset / source / "N_1000"
        gd = load_graphdata(gdir)
        nodes = gd.nodes
        edges = gd.edges

        ax = fig.add_subplot(1, 2, j+1, projection="3d")
        ax.scatter(nodes["x_mpc"], nodes["y_mpc"], nodes["z_mpc"], s=2)
        ax.set_title(f"{preset} | {source} | N=1000")
        ax.set_xlabel("x_mpc"); ax.set_ylabel("y_mpc"); ax.set_zlabel("z_mpc")

    plt.tight_layout()
    plt.show()

    # Degree hist
    fig = plt.figure(figsize=(12, 4))
    for j, source in enumerate(["sdss_dr17", "desi_dr1"]):
        gdir = OUT / "graphs" / "cosmology" / preset / source / "N_1000"
        gd = load_graphdata(gdir)
        G = edges_to_nx(len(gd.nodes), gd.edges)
        deg = np.array([d for _, d in G.degree()], dtype=int)

        ax = fig.add_subplot(1, 2, j+1)
        ax.hist(deg, bins=30)
        ax.set_title(f"Degree hist: {preset} | {source} | mean={deg.mean():.2f}")
        ax.set_xlabel("degree"); ax.set_ylabel("count")

    plt.tight_layout()
    plt.show()


## 6) Что дальше

- В этом ноутбуке получены космологические графы **строго** размеров `N=1000` и `N=10000`.
- Во втором ноутбуке мы:
  1) генерируем **квантовые ансамбли** на тех же N и с контролируемыми параметрами (W, p);  
  2) извлекаем единый набор признаков;  
  3) сравниваем домены (космология vs квантовые семейства) по структурным и спектральным метрикам.
